In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv', encoding='gb18030')

In [3]:
df.head()

,comment,star
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...,2
1,菜品丰富质量好，服务也不错！很喜欢！,4
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,2
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,5
4,先说我算是娜娜家风荷园开业就一直在这里吃 每次出去回来总想吃一回 有时觉得外面的西式简餐总是...,4


In [4]:
df.shape

(2000, 2)

In [5]:
def make_label(df):
    df["sentiment"] = df["star"].apply(lambda x: 1 if x>3 else 0)

In [6]:
make_label(df)

In [7]:
df.head()

,comment,star,sentiment
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...,2,0
1,菜品丰富质量好，服务也不错！很喜欢！,4,1
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,2,0
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,5,1
4,先说我算是娜娜家风荷园开业就一直在这里吃 每次出去回来总想吃一回 有时觉得外面的西式简餐总是...,4,1


In [8]:
X = df[['comment']]
y = df.sentiment

In [9]:
X.shape

(2000, 1)

In [10]:
y.shape

(2000,)

In [11]:
X.head()

,comment
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...
1,菜品丰富质量好，服务也不错！很喜欢！
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...
4,先说我算是娜娜家风荷园开业就一直在这里吃 每次出去回来总想吃一回 有时觉得外面的西式简餐总是...


In [12]:
import jieba

In [13]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [14]:
X['cutted_comment'] = X.comment.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\shanshan\AppData\Local\Temp\jieba.cache
Loading model cost 1.103 seconds.
Prefix dict has been built succesfully.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
X.cutted_comment[:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [ ]:
stop_words_file = "stopwordsHIT.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [ ]:
stopwords[-10:]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect = CountVectorizer()

In [ ]:
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())

In [ ]:
term_matrix.head()

In [ ]:
term_matrix.shape

In [ ]:
vect = CountVectorizer(stop_words=frozenset(stopwords))

In [ ]:
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())

In [ ]:
term_matrix.head()

In [ ]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。

In [ ]:
vect = CountVectorizer(max_df = max_df, 
                       min_df = min_df, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))

In [ ]:
term_matrix = pd.DataFrame(vect.fit_transform(X_train.cutted_comment).toarray(), columns=vect.get_feature_names())

In [ ]:
term_matrix.head()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(vect, nb)

In [ ]:
pipe.steps

In [ ]:
from sklearn.cross_validation import cross_val_score
cross_val_score(pipe, X_train.cutted_comment, y_train, cv=5, scoring='accuracy').mean()

In [ ]:
pipe.fit(X_train.cutted_comment, y_train)

In [ ]:
pipe.predict(X_test.cutted_comment)

In [ ]:
y_pred = pipe.predict(X_test.cutted_comment)

In [ ]:
from sklearn import metrics

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
metrics.confusion_matrix(y_test, y_pred)

In [ ]:
from snownlp import SnowNLP
def get_sentiment(text):
    return SnowNLP(text).sentiments

In [ ]:
y_pred_snownlp = X_test.comment.apply(get_sentiment)

In [ ]:
y_pred_snownlp_normalized = y_pred_snownlp.apply(lambda x: 1 if x>0.5 else 0)

In [ ]:
y_pred_snownlp_normalized[:5]

In [ ]:
metrics.accuracy_score(y_test, y_pred_snownlp_normalized)

In [ ]:
metrics.confusion_matrix(y_test, y_pred_snownlp_normalized)